In [5]:
# import necessary modules
import numpy as np
import pandas as pd
from random import random, seed

In [2]:
# Initialize Network
def initialize_network(n_inputs, n_hidden, n_outputs):
    # Create network and layers as lists
    network = []
    # Create the hidden and output layer where neurons are saved as dictionaries
    hidden_layer = [{"weights":[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    
    output_layer = [{"weights":[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    
    return network

In [9]:
seed(1)

network = initialize_network(7,2,3)

for i, layer in enumerate(network):
    for k, neuron in enumerate(layer):
        weights = [np.round(x, 8) for x in neuron["weights"]]
        neuron["weights"] = weights
        print(neuron)
    print()

{'weights': [0.13436424, 0.84743374, 0.76377462, 0.25506903, 0.49543509, 0.44949106, 0.65159297, 0.78872335]}
{'weights': [0.09385959, 0.02834748, 0.8357651, 0.43276707, 0.76228008, 0.00210605, 0.44538719, 0.72154003]}

{'weights': [0.22876222, 0.9452707, 0.90142746]}
{'weights': [0.03058998, 0.02544586, 0.54141247]}
{'weights': [0.93914916, 0.38120424, 0.2165994]}

